In [7]:
import sys
sys.path.append('../')
import glob2 as glob
import ujson as json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

pd.options.display.max_columns = 200
pd.options.display.max_rows = 200
%matplotlib inline

import warnings; warnings.simplefilter('ignore')

In [8]:
pd.options.display.max_columns = 500
pd.options.display.max_rows = 500

In [3]:
#this is to make sure we have "ujson" and "tqdm"
try:
    import ujson as json
except ModuleNotFoundError:
    import json
    print ('Please install ujson to read JSON oblects faster')
    
try:
    from tqdm import tqdm_notebook
except ModuleNotFoundError:
    tqdm_notebook = lambda x: x
    print ('Please install tqdm to track progress with Python loops')

In [17]:
# Importing Data
path2data = '../Analysis/data/'

dpc = pd.read_csv(path2data + 'DPC_matches_cleaned.csv') ## import csv of match history and picks

In [18]:
dpc.drop(columns='Unnamed: 0', inplace=True)

In [19]:
dpc.head(30)

match_id  duration  leagueid  match_seq_num  series_id  series_type  \
0   4833593109      4036     10979     4062972494     337111            1   
1   4833371463      2976     10979     4062797533     337111            1   
2   4833120790      2700     10979     4062637540     337111            1   
3   4835973421      3504     10979     4064617551     337330            1   
4   4835092211      2530     10979     4063983045     337299            1   
5   4835349235      2983     10979     4064165158     337299            1   
6   4844403184      3606     10979     4070576815     338176            2   
7   4838510531      3234     10979     4066415388     337601            1   
8   4836456903      2959     10979     4064924831     337366            1   
9   4832621485      2404     10979     4062295592     337088            1   
10  4840125375      3264     10979     4067539200     337770            1   
11  4837606570      3441     10979     4065791023     337520            1   
12  4835583515      2191     10979     4064331442     337330            1   
13  4839981142      2726     10979     4067413081     337770            1   
14  4840900564      2272     10979     4068065670     337862            1   
15  4841677189      2041     10979     4068587816     337932            1   
16  4841117691      2357     10979     4068213283     337862            1   
17  4843909697      2653     10979     4070214423     338176            2   
18  4843550889      2307     10979     4069954027     338141            1   
19  4836366115      1869     10979     4064834441     337366            1   
20  4836235481      1984     10979     4064756349     337366            1   
21  4844103057      1906     10979     4070345485     338176            2   
22  4841320882      2328     10979     4068367708     337862            1   
23  4834063398      2511     10979     4063251261     337161            1   
24  4841541944      2082     10979     4068498796     337932            1   
25  4837947429      2234     10979     4066005383     337539            1   
26  4840339077      1921     10979     4067641090     337770            1   
27  4835803139      2015     10979     4064450826     337330            1   
28  4844267214      2079     10979     4070452967     338176            2   
29  4833860999      2197     10979     4063097949     337161            1   

    patch  dire_team_id  radiant_team_id  radiant_win_x  radiant_win_y  \
0      41     7118032.0         111474.0           True           True   
1      41      111474.0        7118032.0           True           True   
2      41      111474.0        7118032.0          False          False   
3      41     6214973.0        7059982.0          False          False   
4      41     7137328.0        7118032.0           True           True   
5      41     7137328.0        7118032.0          False          False   
6      41      111474.0        6214973.0          False          False   
7      41     7137328.0              4.0           True           True   
8      41      543897.0              3.0          False          False   
9      41           4.0        7137328.0           True           True   
10     41      111474.0        7059982.0          False          False   
11     41      543897.0        6214973.0           True           True   
12     41     7059982.0        6214973.0          False          False   
13     41      111474.0        7059982.0           True           True   
14     41      111474.0        6214973.0           True           True   
15     41     7059982.0        7137328.0          False          False   
16     41      111474.0        6214973.0          False          False   
17     41      111474.0        6214973.0          False          False   
18     41     6214973.0        7059982.0          False          False   
19     41      543897.0              3.0          False          False   
20     41      543897.0              3.0           True         

Woops, guess I doubled up on radiant win columns

In [20]:
for col in dpc.columns:
    print(col, ': ', len(dpc[col].unique()), ' labels')

match_id :  1162  labels
duration :  918  labels
leagueid :  5  labels
match_seq_num :  1162  labels
series_id :  599  labels
series_type :  3  labels
patch :  1  labels
dire_team_id :  106  labels
radiant_team_id :  99  labels
radiant_win_x :  2  labels
radiant_win_y :  2  labels
1is_pick :  1  labels
1team :  2  labels
1hero_id :  72  labels
2is_pick :  1  labels
2team :  2  labels
2hero_id :  69  labels
3is_pick :  1  labels
3team :  2  labels
3hero_id :  78  labels
4is_pick :  1  labels
4team :  2  labels
4hero_id :  71  labels
5is_pick :  1  labels
5team :  2  labels
5hero_id :  79  labels
6is_pick :  1  labels
6team :  2  labels
6hero_id :  75  labels
7is_pick :  1  labels
7team :  2  labels
7hero_id :  59  labels
8is_pick :  1  labels
8team :  2  labels
8hero_id :  81  labels
9is_pick :  1  labels
9team :  2  labels
9hero_id :  89  labels
10is_pick :  1  labels
10team :  2  labels
10hero_id :  97  labels
11is_pick :  1  labels
11team :  2  labels
11hero_id :  112  labels
12is_pi

In [6]:
#dpc.isna().sum()
dpc.fillna(0)

match_id  duration  leagueid  match_seq_num  series_id  series_type  \
0     4833593109      4036     10979     4062972494     337111            1   
1     4833371463      2976     10979     4062797533     337111            1   
2     4833120790      2700     10979     4062637540     337111            1   
3     4835973421      3504     10979     4064617551     337330            1   
4     4835092211      2530     10979     4063983045     337299            1   
...          ...       ...       ...            ...        ...          ...   
1157  5114927085       887     11280     4291834666     386573            1   
1158  5114498359      1365     11280     4291475848     386464            1   
1159  5056346258       988     11280     4241637919     374084            1   
1160  5060704777      1137     11280     4245399407     375084            1   
1161  5055077558      1223     11280     4240590934     373748            1   

      patch  dire_team_id  radiant_team_id  radiant_win_x  radiant_win_y  \
0        41     7118032.0         111474.0           True           True   
1        41      111474.0        7118032.0           True           True   
2        41      111474.0        7118032.0          False          False   
3        41     6214973.0        7059982.0          False          False   
4        41     7137328.0        7118032.0           True           True   
...     ...           ...              ...            ...            ...   
1157     41     7409177.0        6209143.0           True           True   
1158     41     7121518.0        2108395.0           True           True   
1159     41     6214973.0        7424508.0          False          False   
1160     41     7441136.0             67.0           True           True   
1161     41     7404529.0        2108395.0           True           True   

      1is_pick  1team  1hero_id  2is_pick  2team  2hero_id  3is_pick  3team  \
0         True  False        38      True   True       111      True  False   
1         True  False       111      True   True        38      True  False   
2         True  False        37      True   True        38      True  False   
3         True  False        66      True   True        52      True  False   
4         True   True        38      True  False        65      True   True   
...        ...    ...       ...       ...    ...       ...       ...    ...   
1157      True  False        85      True   True         7      True  False   
1158      True  False        59      True   True        66      True  False   
1159      True   True        97      True  False        72      True   True   
1160      True  False       102      True   True        86      True  False   
1161      True  False        19      True   True        60      True  False   

      3hero_id  4is_pick  4team  4hero_id  5is_pick  5team  5hero_id  \
0           55      True   True       107      True  False        46   
1           37      True   True        55      True  False        88   
2          111      True   True        55      True  False        88   
3          120      True   True        88      True  False        55   
4           66      True  False       111      True   True        88   
...        ...       ...    ...       ...       ...    ...       ...   
1157        90      True   True        72      True  False         9   
1158        34      True   True         7      True  False        57   
1159        84      True  False        66      True   True        79   
1160       100      True   True        25      True  False        97   
1161         3      True   True        72      True  False        79   

      6is_pick  6team  6hero_id  7is_pick  7team  7hero_id  8is_pick  8team  \
0         True   True        66      True  False        18      True   True   
1         True   True        66      True  False        16      True   True   
2         True   True        66      True  False        16      True   True   
3         True  

In [7]:
dpc.head()

match_id  duration  leagueid  match_seq_num  series_id  series_type  \
0  4833593109      4036     10979     4062972494     337111            1   
1  4833371463      2976     10979     4062797533     337111            1   
2  4833120790      2700     10979     4062637540     337111            1   
3  4835973421      3504     10979     4064617551     337330            1   
4  4835092211      2530     10979     4063983045     337299            1   

   patch  dire_team_id  radiant_team_id  radiant_win_x  radiant_win_y  \
0     41     7118032.0         111474.0           True           True   
1     41      111474.0        7118032.0           True           True   
2     41      111474.0        7118032.0          False          False   
3     41     6214973.0        7059982.0          False          False   
4     41     7137328.0        7118032.0           True           True   

   1is_pick  1team  1hero_id  2is_pick  2team  2hero_id  3is_pick  3team  \
0      True  False        38      True   True       111      True  False   
1      True  False       111      True   True        38      True  False   
2      True  False        37      True   True        38      True  False   
3      True  False        66      True   True        52      True  False   
4      True   True        38      True  False        65      True   True   

   3hero_id  4is_pick  4team  4hero_id  5is_pick  5team  5hero_id  6is_pick  \
0        55      True   True       107      True  False        46      True   
1        37      True   True        55      True  False        88      True   
2       111      True   True        55      True  False        88      True   
3       120      True   True        88      True  False        55      True   
4        66      True  False       111      True   True        88      True   

   6team  6hero_id  7is_pick  7team  7hero_id  8is_pick  8team  8hero_id  \
0   True        66      True  False        18      True   True        16   
1   True        66      True  False        16      True   True       102   
2   True        66      True  False        16      True   True        18   
3   True        37      True  False        16      True   True       102   
4  False        46      True   True        37      True  False        68   

   9is_pick  9team  9hero_id  10is_pick  10team  10hero_id  11is_pick  11team  \
0      True   True        37       True   False         65       True   False   
1      True   True        10       True   False         46       True   False   
2      True   True       121       True   False         46       True   False   
3      True   True        86       True   False         53       True   False   
4      True  False        16       True    True         52       True    True   

   11hero_id  12is_pick  12team  12hero_id  13is_pick  13team  13hero_id  \
0         57       True    True         88       True   False         39   
1         65       True    True         68       True   False        107   
2         65       True    True        114       True   False         57   
3         18       True    True         89       True   False        106   
4         86       True   False          7       True    True        106   

   14is_pick  14team  14hero_id  15is_pick  15team  15hero_id  16is_pick  \
0       True    True        121       True    True         67       True   
1       True    True         18       True    True        119       True   
2       True    True         95       True    True        107       True   
3       True    True         47       True    True         99       True   
4       True   False         82       True   False         42       True   

   16team  16hero_id  17is_pick  17team  17hero_id  18is_pick  18team  \
0   False         68       True    True          7       True   False   
1   False          5       True    True         23       True   False   
2   False         86       True    True         17       True   False   
3   False         68       

## Getting into it 

Recall Labels: 

- 'Team' = 0 = Radiant; 1=Dire
- Picks: ([i]is_pick)
    - First Phase: (1-10)
        - 1-6 BANS
        - 7-10 PICKS 
    - Second Phase: (11-18)
        - 11-14 B
        - 15-18 P
    - Third Phase: (19-22)
        - 19-20 B
        - 21-22 P

Subset cleaned data to get  match_id + radiant_win + drafts

In [8]:
#picks_cols = [str(i) for i in range(5) + 'hero_id']
#picks_ = dpc.iloc[:,[1,2,10:]]

In [9]:
picks = dpc.iloc[:, np.r_[0:2,10:77]]
picks = picks.rename(columns={'radiant_win_y': 'radiant_win'})

In [10]:
### Probably not the best way to do this, but I already had this snipped saved from when I was cleaning the data, so let's run with it. 

df = picks[['match_id', 'radiant_win', 'duration', 
                '1is_pick', '1team', '1hero_id',
                '2is_pick', '2team', '2hero_id', 
                '3is_pick', '3team', '3hero_id', 
                '4is_pick', '4team', '4hero_id', 
                '5is_pick', '5team', '5hero_id', 
                '6is_pick', '6team', '6hero_id', 
                '7is_pick', '7team', '7hero_id', 
                '8is_pick', '8team', '8hero_id', 
                '9is_pick', '9team', '9hero_id', 
                '10is_pick', '10team', '10hero_id', 
                '11is_pick', '11team', '11hero_id', 
                '12is_pick', '12team', '12hero_id',
                '13is_pick', '13team', '13hero_id', 
                '14is_pick', '14team', '14hero_id', 
                '15is_pick', '15team', '15hero_id', 
                '16is_pick', '16team', '16hero_id', 
                '17is_pick', '17team', '17hero_id',
                '18is_pick', '18team', '18hero_id', 
                '19is_pick', '19team', '19hero_id',
                '20is_pick', '20team', '20hero_id',
                '21is_pick', '21team', '21hero_id', 
                '22is_pick', '22team', '22hero_id']]

In [11]:
df

match_id  radiant_win  duration  1is_pick  1team  1hero_id  2is_pick  \
0     4833593109         True      4036      True  False        38      True   
1     4833371463         True      2976      True  False       111      True   
2     4833120790        False      2700      True  False        37      True   
3     4835973421        False      3504      True  False        66      True   
4     4835092211         True      2530      True   True        38      True   
...          ...          ...       ...       ...    ...       ...       ...   
1157  5114927085         True       887      True  False        85      True   
1158  5114498359         True      1365      True  False        59      True   
1159  5056346258        False       988      True   True        97      True   
1160  5060704777         True      1137      True  False       102      True   
1161  5055077558         True      1223      True  False        19      True   

      2team  2hero_id  3is_pick  3team  3hero_id  4is_pick  4team  4hero_id  \
0      True       111      True  False        55      True   True       107   
1      True        38      True  False        37      True   True        55   
2      True        38      True  False       111      True   True        55   
3      True        52      True  False       120      True   True        88   
4     False        65      True   True        66      True  False       111   
...     ...       ...       ...    ...       ...       ...    ...       ...   
1157   True         7      True  False        90      True   True        72   
1158   True        66      True  False        34      True   True         7   
1159  False        72      True   True        84      True  False        66   
1160   True        86      True  False       100      True   True        25   
1161   True        60      True  False         3      True   True        72   

      5is_pick  5team  5hero_id  6is_pick  6team  6hero_id  7is_pick  7team  \
0         True  False        46      True   True        66      True  False   
1         True  False        88      True   True        66      True  False   
2         True  False        88      True   True        66      True  False   
3         True  False        55      True   True        37      True  False   
4         True   True        88      True  False        46      True   True   
...        ...    ...       ...       ...    ...       ...       ...    ...   
1157      True  False         9      True   True        97      True  False   
1158      True  False        57      True   True        76      True  False   
1159      True   True        79      True  False        60      True   True   
1160      True  False        97      True   True        41      True  False   
1161      True  False        79      True   True        84      True  False   

      7hero_id  8is_pick  8team  8hero_id  9is_pick  9team  9hero_id  \
0           18      True   True        16      True   True        37   
1           16      True   True       102      True   True        10   
2           16      True   True        18      True   True       121   
3           16      True   True       102      True   True        86   
4           37      True  False        68      True  False        16   
...        ...       ...    ...       ...       ...    ...       ...   
1157        66      True   True        81      True   True        25   
1158        97      True   True        90      True   True         8   
1159        23      True  False        85      True  False        25   
1160        66      True   True        90      True   True        57   
1161        90      True   True        23      True   True        25   

      10is_pick  10team  10hero_id  11is_pick  11team  11hero_id  12is_pick  \
0          True   False         65       True   False         57       True   
1          True   False         46       True   False         65       True   
2          True   False         46       True  

In [12]:
df= df.astype('int')
df

match_id  radiant_win  duration  1is_pick  1team  1hero_id  2is_pick  \
0     4833593109            1      4036         1      0        38         1   
1     4833371463            1      2976         1      0       111         1   
2     4833120790            0      2700         1      0        37         1   
3     4835973421            0      3504         1      0        66         1   
4     4835092211            1      2530         1      1        38         1   
...          ...          ...       ...       ...    ...       ...       ...   
1157  5114927085            1       887         1      0        85         1   
1158  5114498359            1      1365         1      0        59         1   
1159  5056346258            0       988         1      1        97         1   
1160  5060704777            1      1137         1      0       102         1   
1161  5055077558            1      1223         1      0        19         1   

      2team  2hero_id  3is_pick  3team  3hero_id  4is_pick  4team  4hero_id  \
0         1       111         1      0        55         1      1       107   
1         1        38         1      0        37         1      1        55   
2         1        38         1      0       111         1      1        55   
3         1        52         1      0       120         1      1        88   
4         0        65         1      1        66         1      0       111   
...     ...       ...       ...    ...       ...       ...    ...       ...   
1157      1         7         1      0        90         1      1        72   
1158      1        66         1      0        34         1      1         7   
1159      0        72         1      1        84         1      0        66   
1160      1        86         1      0       100         1      1        25   
1161      1        60         1      0         3         1      1        72   

      5is_pick  5team  5hero_id  6is_pick  6team  6hero_id  7is_pick  7team  \
0            1      0        46         1      1        66         1      0   
1            1      0        88         1      1        66         1      0   
2            1      0        88         1      1        66         1      0   
3            1      0        55         1      1        37         1      0   
4            1      1        88         1      0        46         1      1   
...        ...    ...       ...       ...    ...       ...       ...    ...   
1157         1      0         9         1      1        97         1      0   
1158         1      0        57         1      1        76         1      0   
1159         1      1        79         1      0        60         1      1   
1160         1      0        97         1      1        41         1      0   
1161         1      0        79         1      1        84         1      0   

      7hero_id  8is_pick  8team  8hero_id  9is_pick  9team  9hero_id  \
0           18         1      1        16         1      1        37   
1           16         1      1       102         1      1        10   
2           16         1      1        18         1      1       121   
3           16         1      1       102         1      1        86   
4           37         1      0        68         1      0        16   
...        ...       ...    ...       ...       ...    ...       ...   
1157        66         1      1        81         1      1        25   
1158        97         1      1        90         1      1         8   
1159        23         1      0        85         1      0        25   
1160        66         1      1        90         1      1        57   
1161        90         1      1        23         1      1        25   

      10is_pick  10team  10hero_id  11is_pick  11team  11hero_id  12is_pick  \
0             1       0         65          1       0         57          1   
1             1       0         46          1       0         65          1   
2             1       0         46          1  

In [13]:
# Cleaned data set with all values converted to integers.
df.to_csv('data/DPC_converted.csv')

In [125]:
df1 = pd.read_csv('data/DPC_converted.csv', index_col=False)

In [126]:
df1.drop(columns='Unnamed: 0', inplace=True)

In [101]:
df1.set_index('match_id', inplace=True)

In [104]:
df1.head()

radiant_win  duration  1is_pick  1team  1hero_id  2is_pick  2team  \
match_id                                                                        
4833593109            1      4036         1      0        38         1      1   
4833371463            1      2976         1      0       111         1      1   
4833120790            0      2700         1      0        37         1      1   
4835973421            0      3504         1      0        66         1      1   
4835092211            1      2530         1      1        38         1      0   

            2hero_id  3is_pick  3team  3hero_id  4is_pick  4team  4hero_id  \
match_id                                                                     
4833593109       111         1      0        55         1      1       107   
4833371463        38         1      0        37         1      1        55   
4833120790        38         1      0       111         1      1        55   
4835973421        52         1      0       120         1      1        88   
4835092211        65         1      1        66         1      0       111   

            5is_pick  5team  5hero_id  6is_pick  6team  6hero_id  7is_pick  \
match_id                                                                     
4833593109         1      0        46         1      1        66         1   
4833371463         1      0        88         1      1        66         1   
4833120790         1      0        88         1      1        66         1   
4835973421         1      0        55         1      1        37         1   
4835092211         1      1        88         1      0        46         1   

            7team  7hero_id  8is_pick  8team  8hero_id  9is_pick  9team  \
match_id                                                                  
4833593109      0        18         1      1        16         1      1   
4833371463      0        16         1      1       102         1      1   
4833120790      0        16         1      1        18         1      1   
4835973421      0        16         1      1       102         1      1   
4835092211      1        37         1      0        68         1      0   

            9hero_id  10is_pick  10team  10hero_id  11is_pick  11team  \
match_id                                                                
4833593109        37          1       0         65          1       0   
4833371463        10          1       0         46          1       0   
4833120790       121          1       0         46          1       0   
4835973421        86          1       0         53          1       0   
4835092211        16          1       1         52          1       1   

            11hero_id  12is_pick  12team  12hero_id  13is_pick  13team  \
match_id                                                                 
4833593109         57          1       1         88          1       0   
4833371463         65          1       1         68          1       0   
4833120790         65          1       1        114          1       0   
4835973421         18          1       1         89          1       0   
4835092211         86          1       0          7          1       1   

            13hero_id  14is_pick  14team  14hero_id  15is_pick  15team  \
match_id                                                                 
4833593109         39          1       1        121          1       1   
4833371463        107          1       1         18          1       1   
4833120790         57          1       1         95          1       1   
4835973421        106          1       1         47          1       1   
4835092211        106          1       0         82          1       0   

            15hero_id  16is_pick  16team  16hero_id  17is_pick  17team  \
match_id                                                                 
4833593109         67          1       0         68          1       1   
4833371463        119          1       0          5 

In [172]:
df1.to_csv('data/DPC_pre_split.csv', index=False)

## Setting up prediction problem 

- Target is --> radiant_win

Leading up to this, I dropped the duration columns, as that information would not be known during the drafting phase of a match, only after the match is completed. 

For now I'll just go with radiant_win as my label (should I maybe include duration as a label?), and the drafting columns as my features. 

Let's hot encode and train/test split the data. Since we ended up having to drop the online matches from the first minor and major events, because they were played on patch 7.21, we are left with 1162 matches from our original ~1500, it follows that we will have to do a smaller train test split. Therefore, going forward, we will go with a 90-10 split (originally I was hoping for an 80-20 split). 

In [173]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.model_selection import ShuffleSplit, KFold

In [195]:
data = pd.read_csv((path2data + 'DPC_pre_split.csv'), index_col='match_id')

In [177]:
data.head()

radiant_win  duration  1is_pick  1team  1hero_id  2is_pick  2team  \
match_id                                                                        
4833593109            1      4036         1      0        38         1      1   
4833371463            1      2976         1      0       111         1      1   
4833120790            0      2700         1      0        37         1      1   
4835973421            0      3504         1      0        66         1      1   
4835092211            1      2530         1      1        38         1      0   

            2hero_id  3is_pick  3team  3hero_id  4is_pick  4team  4hero_id  \
match_id                                                                     
4833593109       111         1      0        55         1      1       107   
4833371463        38         1      0        37         1      1        55   
4833120790        38         1      0       111         1      1        55   
4835973421        52         1      0       120         1      1        88   
4835092211        65         1      1        66         1      0       111   

            5is_pick  5team  5hero_id  6is_pick  6team  6hero_id  7is_pick  \
match_id                                                                     
4833593109         1      0        46         1      1        66         1   
4833371463         1      0        88         1      1        66         1   
4833120790         1      0        88         1      1        66         1   
4835973421         1      0        55         1      1        37         1   
4835092211         1      1        88         1      0        46         1   

            7team  7hero_id  8is_pick  8team  8hero_id  9is_pick  9team  \
match_id                                                                  
4833593109      0        18         1      1        16         1      1   
4833371463      0        16         1      1       102         1      1   
4833120790      0        16         1      1        18         1      1   
4835973421      0        16         1      1       102         1      1   
4835092211      1        37         1      0        68         1      0   

            9hero_id  10is_pick  10team  10hero_id  11is_pick  11team  \
match_id                                                                
4833593109        37          1       0         65          1       0   
4833371463        10          1       0         46          1       0   
4833120790       121          1       0         46          1       0   
4835973421        86          1       0         53          1       0   
4835092211        16          1       1         52          1       1   

            11hero_id  12is_pick  12team  12hero_id  13is_pick  13team  \
match_id                                                                 
4833593109         57          1       1         88          1       0   
4833371463         65          1       1         68          1       0   
4833120790         65          1       1        114          1       0   
4835973421         18          1       1         89          1       0   
4835092211         86          1       0          7          1       1   

            13hero_id  14is_pick  14team  14hero_id  15is_pick  15team  \
match_id                                                                 
4833593109         39          1       1        121          1       1   
4833371463        107          1       1         18          1       1   
4833120790         57          1       1         95          1       1   
4835973421        106          1       1         47          1       1   
4835092211        106          1       0         82          1       0   

            15hero_id  16is_pick  16team  16hero_id  17is_pick  17team  \
match_id                                                                 
4833593109         67          1       0         68          1       1   
4833371463        119          1       0          5 

In [160]:
#data.drop(columns='Unnamed: 0', inplace=True)

In [178]:
data.shape

(1162, 68)

In [179]:
def hotencode(data):
    unq, idx = np.unique(data, return_inverse=1)
    col_idx = idx.reshape(data.shape)
    out = np.zeros((len(col_idx),col_idx.max()+1),dtype=int)
    out[np.arange(len(col_idx))[:,None], col_idx] = 1
    return pd.DataFrame(out, columns=unq, index=data.index)

Fuck I'm stupid, we're predicting a classification questions (win/notwin) --> can't have both 'radiant_win' and 'duration' as targets (I think).
But just incase duration might matter, or we want to explore it later, keep it in and simply drop it when we predict radiant_win. 

Targets = (radiant_win, duration); Features = (everthing else)

In [180]:
data

radiant_win  duration  1is_pick  1team  1hero_id  2is_pick  2team  \
match_id                                                                        
4833593109            1      4036         1      0        38         1      1   
4833371463            1      2976         1      0       111         1      1   
4833120790            0      2700         1      0        37         1      1   
4835973421            0      3504         1      0        66         1      1   
4835092211            1      2530         1      1        38         1      0   
...                 ...       ...       ...    ...       ...       ...    ...   
5114927085            1       887         1      0        85         1      1   
5114498359            1      1365         1      0        59         1      1   
5056346258            0       988         1      1        97         1      0   
5060704777            1      1137         1      0       102         1      1   
5055077558            1      1223         1      0        19         1      1   

            2hero_id  3is_pick  3team  3hero_id  4is_pick  4team  4hero_id  \
match_id                                                                     
4833593109       111         1      0        55         1      1       107   
4833371463        38         1      0        37         1      1        55   
4833120790        38         1      0       111         1      1        55   
4835973421        52         1      0       120         1      1        88   
4835092211        65         1      1        66         1      0       111   
...              ...       ...    ...       ...       ...    ...       ...   
5114927085         7         1      0        90         1      1        72   
5114498359        66         1      0        34         1      1         7   
5056346258        72         1      1        84         1      0        66   
5060704777        86         1      0       100         1      1        25   
5055077558        60         1      0         3         1      1        72   

            5is_pick  5team  5hero_id  6is_pick  6team  6hero_id  7is_pick  \
match_id                                                                     
4833593109         1      0        46         1      1        66         1   
4833371463         1      0        88         1      1        66         1   
4833120790         1      0        88         1      1        66         1   
4835973421         1      0        55         1      1        37         1   
4835092211         1      1        88         1      0        46         1   
...              ...    ...       ...       ...    ...       ...       ...   
5114927085         1      0         9         1      1        97         1   
5114498359         1      0        57         1      1        76         1   
5056346258         1      1        79         1      0        60         1   
5060704777         1      0        97         1      1        41         1   
5055077558         1      0        79         1      1        84         1   

            7team  7hero_id  8is_pick  8team  8hero_id  9is_pick  9team  \
match_id                                                                  
4833593109      0        18         1      1        16         1      1   
4833371463      0        16         1      1       102         1      1   
4833120790      0        16         1      1        18         1      1   
4835973421      0        16         1      1       102         1      1   
4835092211      1        37         1      0        68         1      0   
...           ...       ...       ...    ...       ...       ...    ...   
5114927085      0        66         1      1        81         1      1   
5114498359      0        97         1      1        90         1      1   
5056346258      1        23         1      0        85         1      0   
5060704777      0        66         1      1        90         1      1   
5055077558      0        90 

In [196]:
y = data[['radiant_win', 'duration']]
X = data.drop(columns=['radiant_win', 'duration'])

In [197]:
y.shape, X.shape

((1162, 2), (1162, 66))

Splitting data (90% train, 10% test, random_state=42) and saving test target and features dataframes to csv

In [198]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)

In [199]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1045, 66), (117, 66), (1045, 2), (117, 2))

## MADE SECOND SET OF TRAIN TEST AS PRECAUTION

- Also cause I forgot to save train data from first split
- Not 100% on whether i need to do encoding before or after split
    - but based on kaggle examples having data pre-split, and then encoding both train/test, I think it's fine to do after the split

In [202]:
np.bincount(y['radiant_win'])

array([596, 566])

In [200]:
## DONE @ 3am Sat march 22
X_train.to_csv('data_train_test/dpc_train_features.csv', index='match_id')
y_train.to_csv('data_train_test/dpc_train_target.csv', index='match_id')
X_test.to_csv('data_train_test/dpc_test_features.csv', index='match_id')
y_test.to_csv('data_train_test/dpc_test_target.csv', index='match_id')


### DONT TOUCH THESE NOW!!!!!!!!!!!!!

Test features and targets saved to csv @ approx 1:45 am 


In [187]:
X_test.to_csv('data/dpc_test_features.csv', index='match_id')
y_test.to_csv('data/dpc_test_target.csv', index='match_id')

In [191]:
X_test.shape, y_test.shape

((117, 66), (117, 2))

### Train/Validation Split 

Now, we turn our attention to the training set. 

Splitting training data into training and validation sets. Set validation set size to be 0.1, random state as 42 

In [192]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state =42)

In [193]:
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((940, 66), (105, 66), (940, 2), (105, 2))

In [194]:
X_train.to_csv('data/dpc_train_features.csv', index='match_id')
y_train.to_csv('data/dpc_train_target.csv', index='match_id')
X_valid.to_csv('data/dpc_valid_features.csv', index='match_id')
y_valid.to_csv('data/dpc_valid_target.csv', index='match_id')

## FOLLOW UP QUESTIONS/CONSIDERATIONS

- should i have done onehotencoding before or after?
- 